In [1]:
import time
import pandas as pd

import bernielib as bl

In [2]:
bl.listSerialPorts()

['COM7', 'COM18', 'COM20']

In [3]:
ber = bl.robot(cartesian_port_name='COM18', pipette_port_name='COM7', misc_port_name='COM20')

In [6]:
ber.robotHome()
ber.pipetteHome()

In [55]:
ber.moveToWell(rack_name='tips', column=0, row=0, save_height=12)

(27.9, 119.05, 156.9)

In [71]:
ber.moveAxisDelta('z', -50)

In [25]:
ber.moveDownUntilPress(0.5, 200)

159.4

In [27]:
ber.moveDownUntilPress(0.5, 200)

165.4

In [30]:
ber.tips_rack.getZ()

168.9

In [31]:
168.9-159.4

9.5

In [32]:
168.9-165.4

3.5

In [45]:
ber.getPosition?

In [56]:
ber.dumpTip()

In [50]:
def tipPickupAttempt(robot, wrong_hit_threshold=7, initial_force=200, final_force=1800):
    Z_start = robot.getPosition(axis='Z')
    Z_calibrated = robot.tips_rack.getZ()
    Z_wrong_hit_abs = Z_calibrated - wrong_hit_threshold
    Z_soft = robot.moveDownUntilPress(1, initial_force)
    if Z_soft < Z_wrong_hit_abs:
        # Robot hit something before the tip; that means it probably missed.
        # Retracting and ejecting tip (just in case)
        robot.move(z=Z_start)
        robot.dumpTip()
        return False
    else:
        # Robot correctly got to the tip, now pressing further so the tip is firmly attached
        robot.moveDownUntilPress(1, final_force)
        return True

In [82]:
def tryTipPickupAtNextPosition(robot, axis, step):
    robot.moveAxisDelta(axis, step)
    tip_picked_up = tipPickupAttempt(robot)
    return tip_picked_up

In [83]:
def lookForTip(robot, min_radius=0.5, max_radius=3):
    
    x_init = robot.getPosition(axis='X')
    y_init = robot.getPosition(axis='Y')
    z_init = robot.getPosition(axis='Z')
    
    radius=min_radius
    while radius < max_radius:
        # 0 deg
        if tryTipPickupAtNextPosition(robot, 'X', radius):
            return True
        # 45 deg
        if tryTipPickupAtNextPosition(robot, 'Y', radius):
            return True
        # 90 deg
        if tryTipPickupAtNextPosition(robot, 'X', -radius):
            return True
        # 135 deg
        if tryTipPickupAtNextPosition(robot, 'X', -radius):
            return True
        # 180 deg
        if tryTipPickupAtNextPosition(robot, 'Y', -radius):
            return True
        # 225 deg
        if tryTipPickupAtNextPosition(robot, 'Y', -radius):
            return True
        # 270 deg
        if tryTipPickupAtNextPosition(robot, 'X', radius):
            return True
        # 315 deg
        if tryTipPickupAtNextPosition(robot, 'X', radius):
            return True
        radius += min_radius
        robot.move(x=x_init, y=y_init, z=z_init)
    return False

In [51]:
is_success = tipPickupAttempt(ber)

In [93]:
def pickUpTip(robot, column, row, fine_approach_dz=11, raise_z=0, raise_dz_with_tip=60):
    # Moving towards the tip
    robot.moveToWell(rack_name='tips', column=column, row=row, save_height=fine_approach_dz)
    # Attempting to pick up
    tip_picked_up = tipPickupAttempt(robot)
    if not tip_picked_up:
        lookForTip(robot)
    x, y, z = robot.getPosition()
    # Moving up with the tip
    robot.moveAxisDelta('Z', -raise_dz_with_tip)
    robot.tip_attached = 1
    # Letting the rack know that the tip was picked from there
    robot.tips_rack.consume(column, row)
    return x, y, z

In [94]:
pickUpTip(ber, 2, 2)

(46.68, 136.83, 169.9)

In [95]:
ber.dumpTipToPosition(2, 2)

In [64]:
ber.tips_rack.next()

(0, 0)

In [11]:
ber.tips_rack.refill()

# Now testing as part of the library

In [5]:
ber.pickUpTip(0, 0)

Tip calibration is off. Please recalibrate tip rack before running another protocol.


(28.4, 119.05, 168.9)

In [6]:
ber.dumpTipToPosition(0, 0)

In [7]:
ber.calibrateRack(rack='tips')

(76.69999999999999, 149.765, 168.4)

In [8]:
ber.move(z=50)

In [9]:
ber.pickUpTip(0, 0)

(27.8, 118.65, 169.4)

In [10]:
ber.dumpTipToPosition(0, 0)

# Cycling through many tips

In [7]:
ber.tips_rack.refill()

In [8]:
for i in range(60):
    ber.pickUpNextTip()
    ber.moveAxisDelta('Z', 30)
    ber.dumpTip()